# Notebook usage of [BoltzTrap2Y](https://boltztrap2y.readthedocs.io/en/latest/)

This code is a fork of the [BoltzTrap2](https://gitlab.com/sousaw/BoltzTraP2/-/tree/public) originated from Georg K. H. Madsen and Jesús Carrete and Matthieu J. Verstraete

 - Changes are only made for constant doping calculations, partially according to [thermodynamic theory of Seebeck coefficients](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.98.224101). The major works include:

   1. revised `interface.py` 
   2. extended `bandlib.py` into `bandlibEXT.py`.
   3. extended `io.py` into `ioEXT.py` 
   4. added ``remesh.py`` for calculations in low temperature (meshed increased to ~100,000 so that the calculated results can be good until a few of 10 K).
   5. improved the computational efficiency on the calculations of the chemical potential of electrons (Fermi levels)
   6. added `plotpng.py` for figure plot.

The following using [jupyter notebook](https://jupyter.org/) demonstrates the code for the calculations of thermoelectric-related properties based on output from VASP for $Ca_3Ru_2O_7$

**If one just wants to have a fast test, just click Runtime->Run all**

**To restart the test, cilck Edit->Clear all outputs**






# $\color{red}{\text{Installation of BoltzTraP2Y}}$

**1. clone of the forked branch of BoltzTrap2 code from gitlab.
Note that "!git" should be used in place of "git" in notebook. The same is for the command "!btp2" below which is to run the package**

In [ ]:
!git clone https://gitlab.com/yiwang62/BoltzTraP2.git

**2. install the development version**

In [ ]:
cd BoltzTraP2

In [ ]:
pip install -e .

In [ ]:
!btp2 dope -h

# $\color{red}{\text{To calculate the thermoelectirc-related properties}}$
This is a demo to calculate/plot of the chemical potentials of electrons based on DOS's derived by mixing DOSCARs for two phases thermoelectric properties for the AFM_b phase based on the calculated results by VASP for Ca2Ru3O7

For the kinetic calculations, all the commands follow those from BoltzTraP2. For more details, check out the [BoltzTraP2 tutorial](https://gitlab.com/sousaw/BoltzTraP2/-/wikis/tutorial).

*Move to the folder where the Example folder resided*

In [ ]:
cd Example/Ca3Ru2O7

BoltzTraP2 represents each electronic band as a trigonometric polynomial. This model must be trained on input data from an electronic structure program in order to determine the coefficients of each polynomial. To complete this step and save the results to a file, we invoke the btp2 frontend in interpolate mode: (for help, try `btp2 interpolate -h`)

In [ ]:
!btp2 interpolate -m 5 AFM_b/

where  `-m` is a critical parameter controlling how fine the interpolation is. In this case, we tell the program to try and sample five irreducible k points for each one that is present in the input. `AFM_b/` is the folder holding major VASP input/output files, i.e., POSCAR, DOSCAR, EIGENVAL, OUTCAR, and vasprun.xml. After a while, the program will finish and save the results to the interpolation.bt2 file.

Our intersst in BoltzTraP2 is to estimate the thermoelectric properties under given charger carrier doping. To do so, we start from the results of the interpolation step and use the **dope** subcommand (for help, try `btp2 dope -h`):

In [ ]:
#!btp2 dope -b 2000 interpolation.bt2 10:310:10 0
!btp2 dope -b 1000 interpolation.bt2 10:310:10 0

Where `-b 2000` is an approach by smoothening the data mesh into a bin containing 2000 points to resolve the often numerical instability problem. This will explore the temperature range from 100 to 300 K in intervals of 10  K and electron doping level of 1.0`x`$10^{19}$ $e/cm^3$, using the default uniform-relaxation-time model. The output includes:

 - file `interpolation.dope.trace` # main output file
 - file `interpolation.dope.dos` # denser DOS
 - file `interpolation.dope.dos.raw` # raw DOS
 - folder `figures/` containing the main plots

 These data can be seen by click the files/folder icon in the left side bar, uncollapse ``BoltzTrap2/Example/Ca2Ru3O7``, then you see them which can be downloaded by right click.

 For the ``interpolation.dope.trace`` file, the collumns are made of

    collum 1, $\mu-E_f(eV)$ - electron chemical potential

    collum 2, $T(K)$ - temperature

    collum 3, $N(e/uc)$ - number of charge carries due to doping per unit cell

    collum 4, $[1/(Ha*uc)]$ - 

    collum 5, $S(V/K)$ - Seebeck coefficients by BTE theory

    collum 6, $\sigma/tau0[1/(ohm\*m\*s)]$ - trace of electrical conductivity

    collum 7, $RH[m^3/C]$ -

    collum 8, $kappae/tau0[W/(m\*K\*s)]$ -

    collum 9, $C_{\mu}[J/(mole-atom*K)]$ - constant voltage heat capacity, i.e., with given chemical potential

    collum 10, $chi[m^3/mol]$ -

    collum 11, $C_{el}[J/(mole-atom*K)]$ - heat capacity with constant number of eletrons. i.e., with given doping

    collum 12, $S_e(V/K)$ - Seebeck coefficients by thermodynamic understanding, see Phys. Re. B, 98 (2018) 224101.

    collum 13, $n_{eff}(e/cm^3)$ - effective carrier concentration

    collum 14, $L(W*ohm/K^2)$ - Lorenz number by thermodynamic understanding

    collum 15, $\sigma_h$ - hole electrical conductivity

    collum 16, $\sigma_e$ - electon electrical conductivity

    collum 17, $n_h(e/cm^3)$ - hole carrier concentration

    collum 18, $n_e(e/cm^3)$ - electron carrier concentration

    collum 19, $N_h$ - testing
    
    collum 20, $N_e$ - testing


interpolation.dope.trace can be viewed by

In [ ]:
cat interpolation.dope.trace

In [ ]:
#compare the Lorenz Numbers calculated by BTE and TH approaches
from IPython.display import Image
Image(filename='figures/T-L.png') 


In [ ]:
#display the plot for charge carrier mobility
from IPython.display import Image
Image(filename='figures/T-M.png') 

**The following is raw python code to plot any raw data**

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

colors=('k', 'b', 'r', 'c', 'm', 'g', 'y')
linestyles = ['-', '--', ':', '-.', '.', ',', 'o', '^', 'v', '<', '>', 's',
              '+', 'x', 'd', '1', '2', '3', '4', 'h', 'p', '|', '_', 'D', 'H']
    
n_lines = len(linestyles)
n_colors = len(colors)

data = np.genfromtxt('interpolation.dope.trace')

def plotXY(x,y,title="xx",xlabel="xx", ylabel="yy", labely="",
           figname="", y1=None, labely1="",x1=None, xrange=None,vline=None):
    plt.rc('font', size=36)
    matplotlib.rcParams['lines.linewidth'] = 2

    fig,ax = plt.subplots()
    fig.set_size_inches(18,12) #set figsize in inches
    ax.plot(x, y, linestyle=linestyles[0], color=colors[0], label=labely)
    if y1 is not None:
      if x1 is None:
        ax.plot(x, y1, linestyle=linestyles[1], color=colors[0], label=labely1)
      else:
        ax.plot(x1, y1, linestyle=linestyles[1], color=colors[0], label=labely1)

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if xrange is not None:
      plt.xlim(xrange)
      lims = plt.gca().get_xlim()
      ii = np.where( (x > lims[0]) &  (x < lims[1]) )[0]
      _ymax = y[ii].max()
      if y1 is not None: _ymax = max(_ymax, y1[ii].max())
      plt.ylim( [0,1.1*_ymax])
    
 
    legend = plt.legend(loc="best")
    try:
      legend.set_draggable(True)
    except AttributeError:
      pass
    if vline is not None:
      plt.axvline(x = vline, color = 'purple', linestyle=':', label = '$E_f')

    if figname!="": plt.savefig(figname)
    plt.show()




*compare the original VASP DOS with the BoltzTrap2 DOS*

In [ ]:
dos_raw = np.genfromtxt('interpolation.dope.dos_raw')
dos = np.genfromtxt('interpolation.dope.dos')

plotXY(dos_raw[:,0],dos_raw[:,1],title="electron density of states",
       xlabel = "$band\ energy$", ylabel=r"$DOS\ (e/eV/cell)$", labely="DOS by VASP", 
       figname="eDOS", x1=dos[:,0], y1=dos[:,1], labely1="DOS by BoltzTraP2",
       xrange=[-0.2,0.2])

*plot the hole/electron mobility*

$$    \mathbf{M}_{h} = \frac{\mathbf{\sigma}_{h}}{ep} $$

$$    \mathbf{M}_{e} = \frac{\mathbf{\sigma}_{e}}{en} $$


In [ ]:
x = data[:,1]
y = data[:,14]/data[:,18]*100
y1 = data[:,15]/data[:,19]*100

plotXY(x,y,title="Mobility",xlabel = "$T (K)$", ylabel=r"$M\,(\tau_0/e)\,[cm^2\,/(V\,s)]$", labely="hole", figname="T-M-pNn", y1=y1, labely1="electron")

**The following compare the calculated Seebeck coefficients between BTE and TE**

In [ ]:
from IPython.display import Image
Image(filename='figures/T-S.png') 

**The following compare the calculated heat capacitiies under constant voltage vs constant doping**

In [ ]:
from IPython.display import Image
Image(filename='figures/T-cv.png') 

**The following plot the effective charge carrier concentration**

$$ n_{eff} = \int_{- \infty}^{\infty}{f(1 - f)\ D\left( \varepsilon \right)d\varepsilon} $$

In [ ]:
from IPython.display import Image
Image(filename='figures/T-n.png') 

*plot the hole/electron carrier concentration*

$$    n = \int_{\varepsilon_{F}}^{+ \infty}{fDd\varepsilon} $$

$$    p = \int_{- \infty}^{\varepsilon_{F}}{fDd\varepsilon} $$


In [ ]:
x = data[:,1]
y = data[:,18]
y1 = data[:,19]

plotXY(x,y,title="Carrier concentration",xlabel = "$T (K)$", ylabel=r"$n\;\left[\mathrm{\left|e\right|\,cm^{-3}}\right]$", labely="hole", figname="T-n-pNn", y1=y1, labely1="electron")

In [ ]:
from IPython.display import Image
Image(filename='figures/T-sigma.png') 

# $\color{red}{\text{To calculate the electronic contributions to the thermodynamic properties}}$


This is a demo to calculate/plot of the chemical potentials of electrons based on DOS's derived by mixing DOSCARs for two phases (AFM_b and AFM_a) calculated by VASP for $Ca_2Ru_3O_7$


1. align the two DOS's to 0 K Fermi energy

$$ D_{AFM\_b}^{'}\left( \varepsilon \right) = D_{AFM\_b}\left( \varepsilon + \varepsilon_{F}^{AFM\_b} \right) $$

$$ D_{AFM\_a}^{'}\left( \varepsilon \right) = D_{AFM\_a}\left( \varepsilon + \varepsilon_{F}^{AFM\_a} \right) $$


2. mix the DOSs

$$ D_{\text{mix}}^{'}\left( \varepsilon \right) = (1 - x)*D_{AFM\_b}^{'}\left( \varepsilon \right)+{x*D}_{AFM\_a}^{'}\left( \varepsilon \right) $$

3. solve the following eqqtion for the chemical potential of electrons stating the conservation of the number of electrons

$$ \int_{- \infty}^{\infty}{fD_{\text{mix}}^{'}\left( \varepsilon \right)d\varepsilon} = \int_{- \infty}^{0}{D_{\text{mix}}^{'}\left( \varepsilon \right)d\varepsilon} $$

under Fermi distribution

$$ f = \frac{1}{e^{\frac{\varepsilon - \mu}{k_{B}T}} + 1} $$


**Set the parameters followed by run/plot**

        beta: initial mixxing coef
        T0: Low temperature limit
        T1: High temperature limit
        nT: Number of temperatures
        nc: Number of compositions
        d0: path of the first phase #AFM-b phase
        d1: path of the second phase #AFM-a phase


In [ ]:
beta=0
T0=00
T1=300
nT=101
nc=11
d0="./AFM_b"
d1="./AFM_a"

# calculate/plot 
import sys
sys.path.insert(0, "../../")
from BoltzTraP2.utilities.dosmixAPI import thermo_run
thermo_run(beta, nT, T0, T1, nc, d0, d1)

## Following codes is plot/compare DOSs between AFM_b and AFM_a

In [ ]:
import os
import numpy as np
def get_vaspDOS(d0):
  with open (os.path.join(d0,"DOSCAR")) as fp:
    lines = fp.readlines()
  tmp = lines[5]
  data_line = tmp[0:32].split(' ') #n_dos >10000, no space left before it in VASP
  data_line.extend(tmp[32:].split(' '))
  # filter out empty spaces
  data_line = [k for k in data_line if k != '']
  n_dos, eFermi = (int(data_line[2]),float(data_line[3])) 
  data = np.genfromtxt(os.path.join(d0,"DOSCAR"), skip_header=6,
      skip_footer=len(lines)-n_dos-6)
  return data[:,0]-eFermi, data[:,1]

e0,dos0 = get_vaspDOS(d0)
e1,dos1 = get_vaspDOS(d1)


plotXY(e0,dos0,
      title="electron density of states",
      xlabel = "$band\ energy$", ylabel=r"$DOS\ (e/eV/cell)$", labely="AFM-b", 
      figname="edos-PBEsol", 
      x1=e1, y1=dos1,labely1="AFM-a",
      xrange=[-0.2,0.2],vline=0)

## Output 
The output is to a file named ``thermo.out`` containing the collumn of $T$   $x$   $\mu$ and etc. To downloand the ``thermo.out``, click the files/folder icon in the left side bar, uncollapse ``BoltzTrap2/Example/Ca2Ru3O7``, then you see ``thermo.out`` which can be downloaded by right click 

# The followings are for ploting properties at given composition of the second phase

## The Lorenz Number
$$ L = \frac{k_{B}}{e^{2}}\frac{C_{el}}{n_{eff}} $$

In [ ]:
x = 0.8
import numpy as np
from BoltzTraP2.utilities.dosmixAPI import plotcom
data = np.genfromtxt('thermo.out')

plotcom(1,6,data,x,title="L",ylabel="Lorenz Number ($W Ω K^2$)")

## The Seebeck Coef.

$$ S_{e} = - \frac{1}{en_{eff}T}\int_{- \infty}^{\infty}{\left( \varepsilon - \mu \right)\left( 1 - f \right)fD(\varepsilon)d\varepsilon} $$

In [ ]:
plotcom(1,7,data,x,title="S_e",ylabel="Seebeck Coefficient (${\mu}V/K$)",filename="S_e")

## The electronic contribution to entropy

$$ S_{el} = {- k}_{B}\int_{}^{}{\left\lbrack flnf + \left( 1 - f \right)\ln\left( 1 - f \right) \right\rbrack D\left( \varepsilon \right)d\varepsilon} $$


In [ ]:
plotcom(1,3,data,x,title="$S_{el}$",ylabel="$S_{el}$ ($J/T/mole-atom$)",filename="S_el")

## The electronic contribution to heat capacity with fixed doping

$$ C_{el} = \frac{1}{k_{B}T^{2}}\left\{ \int_{}^{}{f\left( 1 - f \right)\left\lbrack \varepsilon - \mu\left( T \right) \right\rbrack^{2}}D\left( \varepsilon \right)d\varepsilon - \frac{\left\lbrack \int_{}^{}{f\left( 1 - f \right)\left\lbrack \varepsilon - \mu\left( T \right) \right\rbrack D(\varepsilon)d\varepsilon} \right\rbrack^{2}}{\int_{}^{}{f\left( 1 - f \right)D(\varepsilon)d\varepsilon}} \right\} $$

In [ ]:
plotcom(1,4,data,x,title="$C_{el}$",ylabel="$C_{el}$ ($J/T/mole-atom$)",filename="C_el")

## The electronic contribution to heat capacity with fixed voltage

$$ C_{\mu} = T\left( \frac{\partial S_{el}}{\partial T} \right)_{\mu}\mathrm{=}\frac{1}{k_{B}T^{2}}\left\{ \int_{}^{}{f\left( 1 - f \right)\left\lbrack \varepsilon - \mu\left( T \right) \right\rbrack^{2}}D\left( \varepsilon \right)d\varepsilon \right\} $$

In [ ]:
plotcom(1,5,data,x,title="$C_{\mu}$",ylabel="$C_{\mu}$ ($J/T/mole-atom$)",filename="C_mu")

**view the raw output**

In [ ]:
!cat thermo.out

## The number of effective charger carriers

$$ n_{eff} = \int_{- \infty}^{\infty}{f(1 - f)\ D\left( \varepsilon \right)d\varepsilon} $$


In [ ]:
plotcom(1,8,data,x,title="$n_{eff}$",ylabel="$n_{eff}$ ($e/cm^3$)",filename="n_eff")

## The number of artificial holes

$$ n_{p} = \int_{- \infty}^{E_f}{(1 - f)\ D\left( \varepsilon \right)d\varepsilon} $$

In [ ]:
plotcom(1,9,data,x,title="$n_p$",ylabel="$n_p$ ($e/cm^3$)",filename="n_p")

## The number of artificial electrons
$$ n_{e} = \int_{E_f}^{\infty}{f\ D\left( \varepsilon \right)d\varepsilon} $$

In [ ]:
plotcom(1,10,data,x,title="$n_e$",ylabel="$n_e$ ($e/cm^3$)",filename="n_e")

# $\color{red}{\text{Use your own data}}$

If you want, you can upload your own files using the following codes so that you can run this notebook using your own data

1. zip your folders into a file, say xxx.zip
2. upload the zip file::

  from google.colab import files
  
  files.upload()

3. unzip the file::

  !unzip xxx.zip

In [ ]:
import sys
from google.colab import files
files.upload()
!unzip Ca3Ru2O7LDA.zip #Ca3Ru2O7LDA.zip is the file name of the zipped file

In [ ]:
d0="./LDA_AFM_b"
d1="./LDA_AFM_a"
e0,dos0 = get_vaspDOS(d0)
e1,dos1 = get_vaspDOS(d1)

plotXY(e0,dos0,
      title="electron density of states",
      xlabel = "$band\ energy$", ylabel=r"$DOS\ (e/eV/cell)$", labely="AFM-b", 
      figname="edos-PBEsol", 
      x1=e1, y1=dos1,labely1="AFM-a",
      xrange=[-0.2,0.2],vline=0)

In [ ]:
!btp2 interpolate -m 5 LDA_AFM_b/
!btp2 dope -b 1000 interpolation.bt2 10:310:10 0

In [ ]:
#display the plot for charge carrier mobility
from IPython.display import Image
Image(filename='figures/T-M.png') 